In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from random import *
import itertools 
from scipy.optimize import minimize

In [ ]:
def softmax(beta, value):   
    num = np.exp(value * beta)
    den = np.exp(value * beta).sum()    
    return num / den

In [ ]:
class parameter_fit(object):
    def __init__(self, df, bounds = ((0,1), (0,5)), guess = [0.1,1]):
        self.df = df
        self.bounds = bounds # range for alpha and beta
        self.guess = guess  # guess to aid scipy minimize
        
    def negative_log_likelihood(self, parameter):
        df = self.df
        alpha = parameter[0] # defining separate parameters
        beta = parameter[1]
        value = 0.5 * np.ones(2) # ensuring computes values for both levers/choices. Value starts at 0.5
        
        choices, rewards = df['right_or_left'].values.astype(int), df['reward'].values.astype(int)
        prob_log = 0
        session = df['session'].values.tolist()
        switch = [i for i in range(1,len(session)) if session[i]!=session[i-1] ]
        
        
        for choice, reward in zip(choices, rewards):
            if choice not in switch:
                value[choice] += alpha * (reward - value[choice])  
            else:
                value = 0.5
            prob_log += np.log(softmax(value, beta)[choice])

        return -prob_log

    def minimisation(self):
        bounds = (self.bounds)
        mini = minimize(self.negative_log_likelihood, self.guess,
                     method='L-BFGS-B',
                     bounds=bounds)  # method is optimisation algorithm,
        return mini

    def data_fit(self):
        data_fit = pd.DataFrame()
        for mice in self.df['mouse'].unique().tolist():
            df1 = self.df[self.df['mouse'] == mice]
            df1 = df1.reset_index(drop = True)
            mouse = df1['mouse'].unique().tolist()
            sim = parameter_fit(df1,self.bounds,self.guess)
            output = sim.minimisation()
            data = pd.DataFrame({'mouse':mouse,'alpha':output.x[0],'beta':output.x[1],
                                '- log likelihood':output.fun})
            data_fit = data_fit.append(data)
        return data_fit

In [ ]:
input_df = pd.read_pickle('dlight_data.pkl')
data = input_df

In [ ]:
session = data['session'].values.tolist() 
switch = [i for i in range(1,len(session)) if session[i]!=session[i-1] ]
switch.append(len(session))
block = list(np.diff(switch))
block = [451] + block

In [ ]:
name = [i for i in range(len(block))]
temp = []
for i, x in enumerate(block):
    t = [name[i] for a in range(block[i])]
    temp.append(t)
flat_list = [item for sublist in temp for item in sublist]

In [ ]:
data['mouse'] = flat_list

In [ ]:
model = parameter_fit(data)
fit = model.data_fit()

In [ ]:
def RW_fit(alpha, left_value, right_value, reward, choices):
    if choices == 1: #if right choice
        delta = (reward - right_value)
        right_value += alpha * (reward - right_value)  
        left_value = left_value
    elif choices == 0: #if left choice
        delta = (reward - left_value)
        left_value += alpha * (reward - left_value)     
        right_value = right_value
    return delta, left_value, right_value, 

In [ ]:
def infer(mouse):  #calulating representations of value and rpe for each session
    leftlever = 0.5
    rightlever = 0.5
    delta = []
    left_value = [0.5]
    right_value = [0.5]
    
    alpha = fit[fit['mouse']==mouse]['alpha'].item() 
    choices = data[data['mouse']==mouse]['right_or_left'].values.tolist() 
    session = data[data['mouse']==mouse]['session'].values.tolist() 
    reward = data[data['mouse']==mouse]['reward'].values.tolist()
    transition = data[data['mouse']==mouse]['transition'].values.tolist()
    correct = data[data['mouse']==mouse]['correct'].values.tolist()
    block = [i for i in range(1,len(transition)) if transition[i]!=transition[i-1] ]
    
    for index, lr in enumerate(reward):  
        if index in switch:
            leftlever = 0.5
            rightlever = 0.5
            a, leftlever, rightlever = RW_fit(alpha, leftlever, rightlever, reward[index], choices[index])
            delta.append(a)
            left_value.append(leftlever)
            right_value.append(rightlever)
        else:
            a, leftlever, rightlever = RW_fit(alpha, leftlever, rightlever, reward[index], choices[index])
            delta.append(a)
            left_value.append(leftlever)
            right_value.append(rightlever)
    
    return delta, left_value, right_value, reward, choices, correct, block

In [ ]:
delta, left_value, right_value, reward, choices, correct, block = infer(17)

In [ ]:
left_unrewarded = []
left_rewarded = []
right_unrewarded = []
right_rewarded = []

for index, lr in enumerate(reward):
    if choices[index] == 1: #if right choice
        if reward[index] == 1:
            right_rewarded.append(index)
        elif reward[index]==0:
            right_unrewarded.append(index)
    elif choices[index] == 0: #if left choice
        if reward[index] == 1:
            left_rewarded.append(index)
        elif reward[index]==0:
            left_unrewarded.append(index)

In [ ]:
rr = [110 for i in range(len(right_rewarded))] + [115 for i in range(len(right_unrewarded))] + [-10 for i in range(len(left_rewarded))]+[-15 for i in range(len(left_unrewarded))]
rewarded = right_rewarded + right_unrewarded + left_rewarded + left_unrewarded

In [ ]:
rewarded_df = pd.DataFrame({'rewarded':rewarded,'number':rr})

In [ ]:
new_list = [i * 100 for i in choices]
new_df = pd.DataFrame(new_list,
                      columns =['Choices'])

In [ ]:
new_df['mean'] = new_df['Choices'].rolling(4, min_periods=1).mean()

In [ ]:
ax = new_df['mean'].plot(color = 'k')  #so 0 is left, 100 is right. Left was first correct
plt.xlim(0,278)
plt.ylim(-0.5,100.5)
plt.axvspan(0, block[0], color='C0', alpha=0.5, lw=0)
plt.axvspan(block[0], block[1], color='C1', alpha=0.5, lw=0)
plt.axvspan(block[1], block[2], color='C0', alpha=0.5, lw=0)
plt.axvspan(block[2], block[3], color='C1', alpha=0.5, lw=0)
plt.axvspan(block[3], block[4], color='C0', alpha=0.5, lw=0)
plt.axvspan(block[4], block[5], color='C1', alpha=0.5, lw=0)
plt.axvspan(block[5], block[6], color='C0', alpha=0.5, lw=0)
plt.axvspan(block[6], block[7], color='C1', alpha=0.5, lw=0)
plt.axvspan(block[7], block[8], color='C0', alpha=0.5, lw=0)
plt.axvspan(block[8], block[9], color='C1', alpha=0.5, lw=0)
plt.axvspan(block[9], block[10], color='C0', alpha=0.5, lw=0)
plt.ylabel('% right lever choice', fontsize = 14)
ax.axes.get_xaxis().set_visible(False)
sns.despine()
plt.rcParams["figure.figsize"] = (20,4)
plt.savefig("4choices.png", transparent=True, bbox_inches='tight')

In [ ]:
plt.plot(left_value[:278], color = 'C0') #so left is blue
plt.plot(right_value[:278], color = 'C1')
plt.xlim(0,278)
sns.despine(top=True, right=True, left=False, bottom=True)
plt.ylabel('Action value', fontsize = 14)
plt.rcParams["figure.figsize"] = (20,3)
ax = plt.gca()
ax.get_xaxis().set_visible(False)
plt.savefig("4value.png", transparent=True, bbox_inches='tight')

In [ ]:
plt.plot(delta, color = 'darkgreen')
plt.xlim(0,278)
sns.despine()
plt.ylabel('RPE', fontsize = 14)
plt.xlabel('Trials', fontsize = 14)
plt.rcParams["figure.figsize"] = (20,3)
plt.savefig("4rpe.png", transparent=True, bbox_inches='tight')

In [ ]:
plt.plot(right_rewarded, [110 for i in range(len(right_rewarded))] , 's', color='C1')
plt.plot(right_unrewarded, [115 for i in range(len(right_unrewarded))], 'x', color = 'C1')
plt.plot(left_rewarded, [-10 for i in range(len(left_rewarded))] , 's', color='C0')
plt.plot(left_unrewarded, [-15 for i in range(len(left_unrewarded))] , 'x', color='C0')
plt.xlim(0,278)
plt.ylim(-20,120)
plt.rcParams["figure.figsize"] = (20,5)
sns.despine(top=True, right=True, left=False, bottom=True)
ax = plt.gca()
ax.axes.yaxis.set_visible(False)
ax.get_xaxis().set_visible(False)
plt.savefig("4rewardedtrials.png", transparent=True, bbox_inches='tight')

In [ ]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

legend_elements = [Line2D([0], [0], color='k', lw=2, label='Choice (4 trials running average)'),
                   Patch(facecolor='C1',
                         label='Right lever has higher reward %'),
                   Patch(facecolor='C0',
                         label='Left lever has higher reward %')]

# Create the figure
fig, ax = plt.subplots()
ax.legend(handles=legend_elements, loc='center', frameon=False, prop={'size': 10})
plt.rcParams["figure.figsize"] = (10,5)

plt.show()
plt.savefig("4leg1.png", transparent=True)

In [ ]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

legend_elements = [Line2D([0], [0], color='C1', lw=2, label='Right lever'),
                   Line2D([0], [0], color='C0', lw=2, label='Left lever'),
                   Line2D([0], [0], color='darkgreen', lw=2, label='Reward prediction error')]

# Create the figure
fig, ax = plt.subplots()
lgf = ax.legend(handles=legend_elements, loc='center', frameon=False, prop={'size': 10})
plt.rcParams["figure.figsize"] = (10,5)

plt.show()
plt.savefig("4leg3.png", transparent=True, bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
axes = plt.gca()
axes.plot(right_rewarded, [110 for i in range(len(right_rewarded))] , 's', color='C1', label = 'Right rewarded trial')
axes.plot(right_unrewarded, [115 for i in range(len(right_unrewarded))], 'x', color = 'C1', label = 'Right unrewarded trial')
axes.plot(left_rewarded, [-10 for i in range(len(left_rewarded))] , 's', color='C0', label = 'Left rewarded trial')
axes.plot(left_unrewarded, [-15 for i in range(len(left_unrewarded))] , 'x', color='C0', label = 'Left unrewarded trial')
plt.rcParams["figure.figsize"] = (10,5)
lgd = axes.legend(bbox_to_anchor=(1, 0.8), loc='upper left', frameon=False, prop={'size': 10})
plt.savefig("4leg2.png", transparent=True, bbox_extra_artists=(lgd,), bbox_inches='tight')